# Домашнее задание
1. Переведите содержимое файла purchase_log.txt в словарь purchases вида:
```python
{'1840e0b9d4': 'Продукты', ...}
```



In [7]:
import json

# распарсить файл fname в словарь, где ключ - user_id, 
# а значение - категория покупки.
# Аргументы:
#   fname - имя файла которое парсим. Каждая строка - json: {"user_id": "string", "category": "string"}
#   skip_first_line - пропускать первую строку (если она заголовок). По-умолчанию - True
# Возвращает:
#   {"<user_id>": "<category>", ...}
def parse_purchases(fname, skip_first_line=True):
    res = {}
    with open(fname, 'r') as fin:
        if skip_first_line:
            next(fin)
        for ln in fin:
            j = json.loads(ln)            
            res[j['user_id']] = j['category']
    return res

# распарсить файл fname в словарь, где ключ - user_id, 
# а значение - список категорий покупки у пользователя.
# Аргументы:
#   fname - имя файла которое парсим. Каждая строка - json: {"user_id": "string", "category": "string"}
#   skip_first_line - пропускать первую строку (если она заголовок). По-умолчанию - True
# Возвращает:
#   {"<user_id>": ["<category>",...], ...}
def parse_purchases_arr(fname, skip_first_line=True):
    res = {}
    with open(fname, 'r') as fin:
        if skip_first_line:
            next(fin)
        for ln in fin:
            j = json.loads(ln)  
            if j['user_id'] not in res:
                res[j['user_id']] = []
            res[j['user_id']].append(j['category']) # можно и с уникальностью проверять
            
    return res
    

In [5]:
len(parse_purchases('purchase_log.txt'))

99517

In [8]:
len(parse_purchases_arr('purchase_log.txt')) # это реализация была избыточно для задачи )

99517

2. Для каждого user_id в файле visit_log.csv определите третий столбец с категорией покупки (если покупка была, сам файл visit_log.csv изменять не надо). Запишите в файл funnel.csv визиты из файла visit_log.csv, в которых были покупки с указанием категории.

Учтите условия на данные:
- содержимое purchase_log.txt помещается в оперативную память компьютера
- содержимое visit_log.csv - нет; используйте только построчную обработку этого файла

In [26]:
def head(fname, lines_num = 5):
    print('# first {} lines of "{}"'.format(lines_num, fname))
    with open(fname, 'r') as f:
        for _ in range(lines_num):
            print(next(f).strip())
            
# посмотрим что в visit_log.csv & purchase_log.txt 
head('visit_log.csv')
print()
head('purchase_log.txt')# как будто что-то изменилось)

# first 5 lines of "visit_log.csv"
user_id,source
6450655ae8,other
b4ea53e670,other
0e37347152,other
96064ae9e0,other

# first 5 lines of "purchase_log.txt"
{"user_id": "user_id", "category": "category"}
{"user_id": "1840e0b9d4", "category": "Продукты"}
{"user_id": "4e4f90fcfb", "category": "Электроника"}
{"user_id": "afea8d72fc", "category": "Электроника"}
{"user_id": "373a6055fe", "category": "Бытовая техника"}


In [28]:
# конвертируем visits_fname -> out_fname добавлением категории покупки у пользователя (если такая есть)
def visits_to_funnels(visits_fname, purchases_fname, out_fname, uid_idx = 0):
    users_purchases = parse_purchases(purchases_fname)
    with open(visits_fname, 'r') as fin, open(out_fname, 'w') as fout:
        hdr = next(fin).strip() + ",category"
        fout.write(hdr+"\n")
        for ln in fin:
            flds    = ln.strip().split(",")
            if flds[uid_idx] in users_purchases:
                cat = users_purchases[ flds[uid_idx] ] 
                fout.write(",".join(flds+[ cat ] )+"\n")
    

In [29]:
visits_to_funnels('visit_log.csv', 'purchase_log.txt', '/tmp/funnel.csv')
head('/tmp/funnel.csv')

# first 5 lines of "/tmp/funnel.csv"
user_id,source,category
1840e0b9d4,other,Продукты
4e4f90fcfb,context,Электроника
afea8d72fc,other,Электроника
2824221f38,email,Продукты
